In [1]:
from argparse import Namespace
import random

import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set()

In [2]:
def spawn(P):
    l = max(0, np.random.normal(loc=P.lower_loc, scale=P.lower_scale))
    u = min(255, np.random.normal(loc=P.upper_loc, scale=P.upper_scale))
        
    r = lambda: np.random.randint(l, u)
    rc = lambda: (r(), r(), r())
    colors = [rc() for _ in range(P.num_colored)] + [P.bg_color] * P.num_empty
    
    img  = Image.new('RGB', 2*(P.sprite_canvas_size,))
    draw = ImageDraw.Draw(img)

    for y in range(P.sprite_unit_size):
        for xl in range(P.sprite_unit_size // 2):
            xr = P.sprite_unit_size - xl - 1

            c = random.choice(colors)

            pos = [(xl * P.sprite_unit, y * P.sprite_unit), 
                   ((xl + 1) * P.sprite_unit, (y+1) * P.sprite_unit)]
            draw.rectangle(pos, c)

            pos = [(xr * P.sprite_unit, y * P.sprite_unit), 
                   ((xr + 1) * P.sprite_unit, (y+1) * P.sprite_unit)]
            draw.rectangle(pos, c)

    return img


def generate(P):
    sz = P.sprite_canvas_size + 4 * P.margin
    w = sz * P.sprites_per_line
    h = sz * P.num_lines

    wall = Image.new('RGB', (w, h))

    for i in range(P.num_lines):
        for j in range(P.sprites_per_line):
            
            sprite = spawn(P)

            img = Image.new('RGB', 2*(sz,), color=P.bg_color)
            img.paste(sprite, (P.margin, P.margin))

            wall.paste(img, (j*sz, i*sz))
    
    return wall

In [40]:
P = Namespace()

P.bg_color           = [3*(255,), 3*(int(0x1e),)][0]

P.sprite_canvas_size = 2 ** 8
P.sprite_unit_size   = 2 ** 4
P.sprite_unit        = P.sprite_canvas_size // P.sprite_unit_size
    
P.num_colored = 5
P.num_empty   = 6

P.lower_loc   = 70
P.lower_scale = 5
P.upper_loc   = 230
P.upper_scale = 10
    
# control final canvas
P.sprites_per_line = 8
P.num_lines        = 16
P.margin           = 20

# generate(P)

wall = generate(P)
wall.save(f'./out/space_invaders_light_canvas_{P.sprite_canvas_size}_unit_{P.sprite_unit_size}.png', 'png')